In [24]:
import pandas as pd
import os

df = pd.read_excel('FAQS Preguntas frecuentes Abril 2024-1.xlsx', engine='openpyxl')
columnas = df.columns
df[columnas[0]] = df[columnas[0]].ffill()

In [25]:
df.head(12)

,Catalogo de respuestas frecuentes,PREGUNTA FRECUENTE,Respuesta Chat GPT
0,Portal de Servicios en Línea,¿Cómo accedo al portal de Servicios en Línea?,🌟 ¡Hola! Estamos encantados de que estés aquí....
1,Portal de Servicios en Línea,¿Cómo puedo consultar el Boletín / Boleta de C...,🌟 ¡Hola! Estamos emocionados de poder ayudarte...
2,Portal de Servicios en Línea,¿Cómo puedo consultar mi Horario?,🌟 ¡Hola! ¡Estamos aquí para ayudarte! 😊 Si nec...
3,Portal de Servicios en Línea,¿Cómo puedo recuperar mi contraseña de acceso ...,"🌟 ¡Hola! Si necesitas recuperar tu contraseña,..."
4,Becas,¿Cómo accedo al portal de Becas?,🌟 ¡Hola! Estamos encantados de ayudarte. 😊 Par...
5,Becas,¿Cómo puedo consultar mi dictamen?,📧 Recibirás una notificación en tu correo inst...
6,Becas,¿Cómo registrar un caso especial de beca?,"🎓 Para solicitar una beca por caso especial, s..."
7,Becas,¿Cómo puedo recuperar mi contraseña de acceso ...,"🔒 Si olvidaste tu contraseña, no te preocupes...."
8,Becas,¿Cuál es el promedio mínimo para conservar la ...,"🎓 Para licenciatura, es importante mantener un..."
9,Becas,¿Cuáles son los requisitos del proceso de reno...,📝 El proceso de renovación de beca consta de d...


In [26]:
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)
from langchain.chat_models import ChatOpenAI

In [27]:
import openai

openai.api_key = os.environ.get('OPENAI_API_KEY')
model_name = "gpt-3.5-turbo"

chat = ChatOpenAI(model_name=model_name, temperature=1, max_tokens=150)

def generar_variantes_agente(texto, tipo):
    mensajes = [
        SystemMessage(content="Eres un asistente que ayuda a generar variantes de preguntas y respuestas. Actúa como un estudiante universitario entre 18 y 25 años que escribe MUY mal (errores de ortografía y cohesión), abrevia TODAS las palabras y solo escribe palabras claves sin formar oraciones completas."),
        HumanMessage(content=f"Genera 2 {'variantes muy distintas de esta pregunta' if tipo == 'pregunta' else 'preguntas muy distintas cuya respuesta educada y completa sea esta'}:\n\n{texto}"),
        SystemMessage(content='Solo escribe las preguntas sin enumerarlas separadas por dos saltos de línea')
    ]
    response = chat(mensajes)
    variantes = response.content.strip().split("\n")  # Suponiendo que las variantes están separadas por líneas
    return [variante.strip() for variante in variantes if variante.strip()]

preguntas_benchmark = []
indices_benchmark = []

for index, row in df.iterrows():
    variantes_pregunta = generar_variantes_agente(row['PREGUNTA FRECUENTE'], 'pregunta')
    variantes_respuesta = generar_variantes_agente(row['Respuesta Chat GPT'], 'respuesta')
    
    for variante in variantes_pregunta:
        preguntas_benchmark.append(variante)
        indices_benchmark.append(index)
    
    for variante in variantes_respuesta:
        preguntas_benchmark.append(variante)
        indices_benchmark.append(index)

benchmark = pd.DataFrame({
    'pregunta': preguntas_benchmark,
    'Documento asociado': indices_benchmark
})

print(list(benchmark['pregunta']))


['como entrar portal online servicios?', '¿Cómo hacer para usar Servicios en Internet?', '¿Cómo se accede al portal de servicios de la UVM?', '¿Qué información se necesita para iniciar sesión en el portal?', 'Como consutar calificaciones?', 'Manera de ver calificaciones?', '¿Cómo puedo revisar mi boletín de calificaciones en la plataforma de la universidad?', '¿Cuál es la forma correcta de acceder al portal de servicios escolares para consultar mis calificaciones académicas?', 'como ver mi horario', 'consulta mis clases', 'Cómo puedo ver mi horario de clases?', 'Qué debo hacer para acceder al portal de servicios escolares y consultar mi horario académico?', 'cómo restablecer password on-line user', '¿Dónde debo hacer clic si necesito recuperar mi contraseña?', '¿Qué debo seleccionar y hacer una vez dentro del portal para recuperar mi usuario y contraseña?', 'como entrar portal becas', 'formas poder accesar portal becas', '¿URL para acceder al portal de alumnos?', '¿Qué datos se necesit

In [28]:
# with pd.ExcelWriter('benchmark.xlsx', engine='openpyxl') as writer:
#     benchmark.to_excel(writer, sheet_name='Horarios', index=False)